In [1]:
import os
import sys
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import Subset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from tqdm import tqdm 
import h5py
from pathlib import Path
from torch.utils import data
import librosa

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
class CustomDataset(data.Dataset):

    def __init__(self, file_path, transform=None):
        super().__init__()
        self.train_data_cache = []
        self.test_data_cache = []
        self.transform = transform
        self.label_count = [0,0,0,0,0,0,0]
        # Search for all h5 files
        p = Path(file_path)
        files = p.glob('coinData_normalized')
        for h5dataset_fp in files:
            print(h5dataset_fp)
            with h5py.File(h5dataset_fp.resolve()) as h5_file:
                # Walk through all groups, extracting datasets
                for gname, group in h5_file.items():
                    k = 0
                    j = 0
                    if gname == 'oneCent':
                        label = 0
                    elif gname == 'twoCent':
                        label = 1
                    elif gname == 'fiveCent':
                        label = 2
                    elif gname == 'twentyCent':
                        label = 3
                    elif gname == 'fiftyCent':
                        label = 4
                    elif gname == 'oneEuro':
                        label = 5
                    elif gname == 'twoEuro':
                        label = 6
                    
                    for dname, ds in tqdm(group.items()):
                        
                        arr = np.pad(np.array(ds, dtype=np.float32), (0, max(int(np.ceil(307200 - len(ds))), 0)))
                        if k < 100:
                            for i in np.array_split(arr[:307200], 300):
                                if np.sum(i) != 0:
                                    self.train_data_cache.append([label, torch.tensor(i).unsqueeze(0)])
                            k += 1
                        elif j < 25:
                            for i in np.array_split(arr[:307200], 300):
                                if np.sum(i) != 0:
                                    self.test_data_cache.append([label, torch.tensor(i).unsqueeze(0)])
                            j += 1


    def __getitem__(self, index):
        return self.data_cache[index]

    def get_test_data(self):
        return self.test_data_cache
    
    def get_train_data(self):
        return self.train_data_cache
    
    def __len__(self):
        return len(self.data_cache)

In [4]:
customData = CustomDataset("/home/marcus/Dokumente/munzwurf/")

/home/marcus/Dokumente/munzwurf/coinData_normalized


100%|████████████████████████████████████████| 196/196 [00:01<00:00, 126.41it/s]


In [5]:
print(len(customData.get_test_data()))
print(len(customData.get_train_data()))

44988
195913


In [6]:
'''
train_number = [0,0,0,0,0,0,0]
test_number = [0,0,0,0,0,0,0]
train_data_idx = []
test_data_idx = []
for i in range(len(customData)):
    if train_number[customData[i][0]] < 30000:
        train_data_idx.append(i)
        train_number[customData[i][0]] += 1
    elif test_number[customData[i][0]] < 25*300:
        test_data_idx.append(i)
        test_number[customData[i][0]] += 1
'''

'\ntrain_number = [0,0,0,0,0,0,0]\ntest_number = [0,0,0,0,0,0,0]\ntrain_data_idx = []\ntest_data_idx = []\nfor i in range(len(customData)):\n    if train_number[customData[i][0]] < 30000:\n        train_data_idx.append(i)\n        train_number[customData[i][0]] += 1\n    elif test_number[customData[i][0]] < 25*300:\n        test_data_idx.append(i)\n        test_number[customData[i][0]] += 1\n'

In [7]:
'''
test_labels = np.array([i[0] for i in customData], dtype = int)
train_labels = np.array([customData[i][0] for i in train_data_idx], dtype = int)
'''

'\ntest_labels = np.array([i[0] for i in customData], dtype = int)\ntrain_labels = np.array([customData[i][0] for i in train_data_idx], dtype = int)\n'

In [8]:
train_data = customData.get_train_data()
test_data = customData.get_test_data()
train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True, pin_memory=False, num_workers=4)
test_dataloader  = DataLoader(test_data, batch_size=32, shuffle=True, pin_memory=False, num_workers=4)


In [9]:
'''
label_count = []
for i in range(7):
    label_count.append(np.count_nonzero(train_labels == i))

plt.bar(range(7), label_count)
'''

'\nlabel_count = []\nfor i in range(7):\n    label_count.append(np.count_nonzero(train_labels == i))\n\nplt.bar(range(7), label_count)\n'

In [10]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1) 
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1) 
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1) 
        self.conv4 = nn.Conv1d(64, 128, kernel_size=3, padding=1) 
        self.conv5 = nn.Conv1d(128, 256, kernel_size=3, padding=1) 
        self.conv6 = nn.Conv1d(256, 512, kernel_size=3, padding=1)
        
        
        self.fc1   = nn.Linear(512, 7) 
     
        
        
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)),3)
        #print(x.shape)
        x = F.max_pool1d(F.relu(self.conv2(x)),3)
        #print(x.shape)
        x = F.max_pool1d(F.relu(self.conv3(x)),3)
        #print(x.shape)
        x = F.max_pool1d(F.relu(self.conv4(x)),3)
        #print(x.shape)
        x = F.max_pool1d(F.relu(self.conv5(x)),3)
        x = F.max_pool1d(F.relu(self.conv6(x)),3)
        #print(x.shape)
        x = torch.flatten(x, 1) 
        #print(x.shape)
        x = F.softmax(self.fc1(x), dim=1)

        
        return x

In [11]:
model = Network().to(device)
print(model)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

Network(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv5): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv6): Conv1d(256, 512, kernel_size=(3,), stride=(1,), padding=(1,))
  (fc1): Linear(in_features=512, out_features=7, bias=True)
)
528423


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
def train(dataloader, optimizer, criterion, model):
    model.train()
    running_loss = 0.0
    j = 0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [labels, inputs]
        
        inputs = data[1]
        labels = data[0]        
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 500 == 499:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 500), end=" ")
            running_loss = 0.0
            if ((epoch+1) % 50) == 0:
                print(outputs[1:10], labels[1:10])


In [14]:

def test(dataloader, optimizer, criterion, model):   
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for labels, inputs in dataloader:
            labels, inputs = labels.to(device), inputs.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} ")
                

In [ ]:
for epoch in range(500):  # loop over the dataset multiple times
    train(train_dataloader, optimizer, criterion, model)
    test(test_dataloader, optimizer, criterion, model)
print('Finished Training')




[1,   500] loss: 1.809 Test Error: Accuracy: 42.6%, Avg loss: 1.722968 
[2,   500] loss: 1.546 Test Error: Accuracy: 51.6%, Avg loss: 1.630038 
[3,   500] loss: 1.479 Test Error: Accuracy: 58.2%, Avg loss: 1.580998 
[4,   500] loss: 1.439 Test Error: Accuracy: 59.6%, Avg loss: 1.564224 
[5,   500] loss: 1.410 Test Error: Accuracy: 64.5%, Avg loss: 1.518919 
[6,   500] loss: 1.387 Test Error: Accuracy: 67.3%, Avg loss: 1.488283 
[7,   500] loss: 1.374 Test Error: Accuracy: 67.8%, Avg loss: 1.487830 
[8,   500] loss: 1.365 Test Error: Accuracy: 64.8%, Avg loss: 1.510624 
[9,   500] loss: 1.351 Test Error: Accuracy: 65.9%, Avg loss: 1.497693 
[10,   500] loss: 1.344 Test Error: Accuracy: 68.7%, Avg loss: 1.474451 
[11,   500] loss: 1.330 Test Error: Accuracy: 69.9%, Avg loss: 1.460921 
[12,   500] loss: 1.324 Test Error: Accuracy: 71.2%, Avg loss: 1.449079 
[13,   500] loss: 1.319 Test Error: Accuracy: 71.9%, Avg loss: 1.444130 
[14,   500] loss: 1.314 Test Error: Accuracy: 69.1%, Avg los